In [167]:
import json
import pandas as pd
import requests

In [168]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [169]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [170]:
with open("../shared-folder-gald/keys2.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [171]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [172]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [173]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [174]:
access_token = get_access_token(client_key, client_secret)

In [ ]:
liked_videos = []
for username in usernames[12989:13868]:
    liked_videos.append(get_liked_info(username, access_token))

In [ ]:
combined = {}
for i, username in enumerate(usernames[12989:13868]):
    combined[username] = liked_videos[i]

In [177]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

                username                   id               creator
0     theclassicthrifter  7300139051334815022          kelseymccosh
1     theclassicthrifter  7402199914693807391       lovelife.review
2     theclassicthrifter  7387616089800281387          krischievous
3     theclassicthrifter  7385323975062113579   ms.johnson.teachess
4     theclassicthrifter  7382324503486926126         tracydaughter
...                  ...                  ...                   ...
1642          david_qiru  7475020996420963589            david_qiru
1643          david_qiru  7470608709488610615      javitoplastilina
1644          david_qiru  7470777983830084869      javitoplastilina
1645          david_qiru  7474806259808521478  all_meats_smoked_bbq
1646          david_qiru  7472467742231104773      javitoplastilina

[1647 rows x 3 columns]


In [178]:
likes_df.to_csv("liked_videos.csv", mode="a", index=False, header=False)